In [52]:
# ============================================================
# CELL 1  –  Imports & GPU confirmation
# ============================================================
import os, math, json, random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, backend as K
from tensorflow.keras.callbacks import (
    EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
)
import matplotlib
matplotlib.use('Agg')                           # safe for Kaggle kernel
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score
)
from sklearn.model_selection import train_test_split
from PIL import Image
import warnings; warnings.filterwarnings('ignore')
from tqdm import tqdm
# ── GPU check ──
print(f"TensorFlow : {tf.__version__}")
print(f"GPUs found : {tf.config.list_physical_devices('GPU')}")

# ── Reproducibility seed ──
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

TensorFlow : 2.19.0
GPUs found : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [53]:
import os

# ── TensorFlow / CUDA log suppression ──
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'   # 0=all, 1=INFO, 2=WARNING, 3=ERROR
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda'

# Optional: silence cuDNN / cuBLAS verbosity
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
tf.get_logger().setLevel('ERROR')


In [54]:
# ============================================================
# CELL 2  –  Global constants  (edit only this cell)
# ============================================================

# ── Paths ──
DATA_ROOT  = "/home/sandeshprasai/Projects/Final_Semester_Project/AI_Attendance_System/ai-ml-model/DataSets/processed/ThirdLap/"   # ← adjust if needed
SAVE_DIR   = "../../src/models/ImageNetModel"

os.makedirs(SAVE_DIR , exist_ok = True)

# ── Dataset ──
IMG_SIZE      = 112                  # pixels (square)
NUM_CLASSES   = 540                 # identities
IMGS_PER_ID   = 300                 # balanced
VAL_ID_FRAC   = 0.2                 # 108 IDs for val

# ── Model ──
EMB_DIM       = 512                 # embedding dimension

# ── ArcFace ──
ARC_SCALE     = 64.0                # s
ARC_MARGIN    = 0.35                # m

# ── Training ──
BATCH_SIZE    = 64
EPOCHS        = 35
LR_INIT       = 1e-3
LR_MIN        = 1e-6
WEIGHT_DECAY  = 1e-4
PATIENCE      = 8                  # early-stop patience

print("Constants loaded.")

Constants loaded.


In [55]:
# ============================================================
# CELL 3  –  Identity-disjoint train / val split
# ============================================================

def scan_dataset(root):
    """Return {identity_name: [abs_path, ...]} from a flat folder-of-folders."""
    identity_dict = {}
    for id_name in sorted(os.listdir(root)):
        id_path = os.path.join(root, id_name)
        if not os.path.isdir(id_path):
            continue
        files = [
            os.path.join(id_path, f)
            for f in os.listdir(id_path)
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
        ]
        if files:
            identity_dict[id_name] = files
    return identity_dict

# ── Scan ──
id_dict = scan_dataset(DATA_ROOT)
all_ids = sorted(id_dict.keys())
print(f"Total identities found : {len(all_ids)}")
print(f"Example identity       : {all_ids[0]}  ({len(id_dict[all_ids[0]])} images)")

# ── Split identities (80 / 20) ──
train_ids, val_ids = train_test_split(
    all_ids, test_size=VAL_ID_FRAC, random_state=SEED
)

# ── Build label map only over TRAINING identities ──
#    Val identities get their own contiguous labels starting at 0
train_id2label = {name: i for i, name in enumerate(sorted(train_ids))}
val_id2label   = {name: i for i, name in enumerate(sorted(val_ids))}

# ── Flat file + label lists ──
train_files, train_labels = [], []
for name in train_ids:
    for fp in id_dict[name]:
        train_files.append(fp)
        train_labels.append(train_id2label[name])

val_files, val_labels = [], []
for name in val_ids:
    for fp in id_dict[name]:
        val_files.append(fp)
        val_labels.append(val_id2label[name])

print(f"\nTrain identities : {len(train_ids)}   images : {len(train_files)}")
print(f"Val   identities : {len(val_ids)}   images : {len(val_files)}")

# ── Persist label maps (useful for inference later) ──
with open(os.path.join(SAVE_DIR, 'train_id2label.json'), 'w') as f:
    json.dump(train_id2label, f)
with open(os.path.join(SAVE_DIR, 'val_id2label.json'), 'w') as f:
    json.dump(val_id2label, f)

Total identities found : 540
Example identity       : n000001  (288 images)

Train identities : 432   images : 128934
Val   identities : 108   images : 32247


In [56]:
# ============================================================
# CELL 4 – tf.data pipeline with augmentation
# ============================================================

import math
import tensorflow as tf


def load_image(path, label):
    """Read + decode + resize + normalise to [-1, 1]."""
    raw = tf.io.read_file(path)
    img = tf.image.decode_jpeg(raw, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 127.5 - 1.0   # → [-1, 1]
    return img, label


def augment(img, label):
    """
    Training-only augmentations (TF-native, GPU-friendly).
    Applied per-image inside the tf.data graph.
    """

    # --------------------------------------------------------
    # Random horizontal flip
    # --------------------------------------------------------
    img = tf.image.random_flip_left_right(img)

    # --------------------------------------------------------
    # Random rotation (±15°) via projective transform
    # --------------------------------------------------------
    angle = tf.random.uniform([], -15.0, 15.0) * math.pi / 180.0
    cos_a = tf.math.cos(angle)
    sin_a = tf.math.sin(angle)
    cx    = tf.cast(IMG_SIZE, tf.float32) / 2.0               # 56.0
    cy    = cx

    tx = cx - cx * cos_a - cy * sin_a
    ty = cy + cx * sin_a - cy * cos_a

    transform = tf.stack([
        cos_a,  sin_a, tx,
       -sin_a,  cos_a, ty,
        0.0,    0.0
    ])

    img = tf.raw_ops.ImageProjectiveTransformV3(
        images=tf.expand_dims(img, 0),
        transforms=tf.expand_dims(transform, 0),
        output_shape=[IMG_SIZE, IMG_SIZE],
        interpolation="BILINEAR",
        fill_mode="CONSTANT",
        fill_value=0.0
    )[0]

    # --------------------------------------------------------
    # Colour jitter
    # --------------------------------------------------------
    img = img * 0.5 + 0.5                                      # [-1,1] → [0,1]
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)
    img = tf.image.random_saturation(img, lower=0.8, upper=1.2)
    img = img * 2.0 - 1.0  

    # --------------------------------------------------------
    # Random erasing (p = 0.2)
    # --------------------------------------------------------
    def _erase(image):
        h, w = IMG_SIZE, IMG_SIZE
        eh = tf.random.uniform([], 8, 40, dtype=tf.int32)
        ew = tf.random.uniform([], 8, 40, dtype=tf.int32)
        y0 = tf.random.uniform([], 0, h - eh, dtype=tf.int32)
        x0 = tf.random.uniform([], 0, w - ew, dtype=tf.int32)

        pad_top = tf.ones([y0, w, 3], tf.float32)
        pad_mid = tf.concat([
            tf.ones([eh, x0, 3], tf.float32),
            tf.zeros([eh, ew, 3], tf.float32),
            tf.ones([eh, w - x0 - ew, 3], tf.float32)
        ], axis=1)
        pad_bot = tf.ones([h - y0 - eh, w, 3], tf.float32)

        mask = tf.concat([pad_top, pad_mid, pad_bot], axis=0)
        return image * mask

    img = tf.cond(
        tf.random.uniform([]) < 0.2,
        lambda: _erase(img),
        lambda: img
    )

    # --------------------------------------------------------
    # Random grayscale (p = 0.1)
    # --------------------------------------------------------
    def _to_gray(x):
        gray = tf.image.rgb_to_grayscale(x)
        return tf.concat([gray, gray, gray], axis=-1)

    img = tf.cond(
        tf.random.uniform([]) < 0.1,
        lambda: _to_gray(img),
        lambda: img
    )

    # --------------------------------------------------------
    # Final clip
    # --------------------------------------------------------
    img = tf.clip_by_value(img, -1.0, 1.0)
    return img, label


def build_dataset(file_paths, labels, is_train=True, batch=BATCH_SIZE):
    """Compose the full tf.data pipeline."""
    ds = tf.data.Dataset.from_tensor_slices(
        (tf.constant(file_paths), tf.constant(labels, tf.int32))
    )
    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

    if is_train:
        ds = ds.shuffle(8192, seed=SEED)
        ds = ds.map(augment, num_parallel_calls=tf.data.AUTOTUNE)

    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds


# ------------------------------------------------------------
# Instantiate datasets
# ------------------------------------------------------------
train_ds = build_dataset(train_files, train_labels, is_train=True)
val_ds   = build_dataset(val_files,   val_labels,   is_train=False)

# ------------------------------------------------------------
# Sanity check
# ------------------------------------------------------------
for batch_imgs, batch_labels in train_ds.take(1):
    print(f"Batch shape : {batch_imgs.shape}")
    print(f"Labels     : {batch_labels.shape}")
    print(f"Pixel range: [{batch_imgs.numpy().min():.2f}, "
          f"{batch_imgs.numpy().max():.2f}]")


Batch shape : (64, 112, 112, 3)
Labels     : (64,)
Pixel range: [-1.00, 1.00]


In [58]:
# ============================================================
# CELL 5  –  IR-ResNet-50 backbone (pretrained ImageNet)
# ============================================================

def build_backbone(input_shape=(IMG_SIZE, IMG_SIZE, 3)):
    """
    Returns a Keras Model:
        input  → [B, 112, 112, 3]
        output → [B, 2048]   (after Global Average Pooling)

    Uses ResNet50 with ImageNet weights as the base.
    The top (classification) layers are removed.
    """
    base = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling=None                              # we add our own GAP
    )
    # Global Average Pooling  →  [B, 2048]
    x = tf.keras.layers.GlobalAveragePooling2D()(base.output)
    backbone = tf.keras.Model(inputs=base.input, outputs=x, name='ir_resnet50')
    return backbone

backbone = build_backbone()
backbone.summary(expand_nested=False)
print(f"\nBackbone output shape : {backbone.output_shape}")   # (None, 2048)

2026-02-04 09:24:08.915429: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.0KiB (rounded to 524288)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2026-02-04 09:24:08.915468: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2026-02-04 09:24:08.915473: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 288, Chunks in use: 288. 72.0KiB allocated for chunks. 72.0KiB in use in bin. 52.9KiB client-requested in use in bin.
2026-02-04 09:24:08.915475: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 202, Chunks in use: 202. 101.0KiB allocated for chunks. 101.0KiB in use in bin. 101.0KiB client-requested

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1,1,256,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2] name: 

In [ ]:
# ============================================================
# CELL 6  –  Embedding head  (Dense → BN → L2 Normalise)
# ============================================================

class L2Normalize(tf.keras.layers.Layer):
    """Custom layer: L2-normalises along the last axis."""
    def call(self, x):
        return tf.nn.l2_normalize(x, axis=-1)


def build_embedding_model():
    """
    Full feature-extractor:
        input  → [B, 112, 112, 3]
        output → [B, 512]  (unit-normalised embedding)
    """
    inp = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='face_input')

    x = backbone(inp)                                  # [B, 2048]
    x = tf.keras.layers.Dense(EMB_DIM, use_bias=False, name='fc_embed')(x)
    x = tf.keras.layers.BatchNormalization(name='bn_embed')(x)
    x = L2Normalize(name='l2_norm')(x)                       # unit hypersphere

    emb_model = tf.keras.Model(inputs=inp, outputs=x, name='face_embedding')
    return emb_model

embedding_model = build_embedding_model()
embedding_model.summary()
print(f"\nEmbedding output shape : {embedding_model.output_shape}")  # (None, 512)

In [ ]:
# ============================================================
# CELL 7  –  ArcFace loss  (custom Keras layer)
# ============================================================

class ArcFaceLayer(tf.keras.layers.Layer):
    """
    Additive-Angular-Margin Softmax (ArcFace).

    Args
    ----
    num_classes : int   number of training identities
    emb_dim     : int   embedding dimension (512)
    scale       : float s – logit scale
    margin      : float m – angular margin (radians concept, passed as float)
    """
    def __init__(self, num_classes, emb_dim, scale=64.0, margin=0.5, **kwargs):
        super().__init__(**kwargs)
        self.num_classes = num_classes
        self.emb_dim     = emb_dim
        self.s           = scale
        self.m           = margin
        self.cos_m       = tf.constant(math.cos(margin), dtype=tf.float32)
        self.sin_m       = tf.constant(math.sin(margin), dtype=tf.float32)
        self.th          = tf.constant(math.cos(math.pi - margin), dtype=tf.float32)
        self.mm          = tf.constant(math.sin(math.pi - margin) * margin, dtype=tf.float32)

    def build(self, input_shape):
        self.W = self.add_weight(
            name='arcface_W',
            shape=(self.num_classes, self.emb_dim),
            initializer='glorot_uniform',
            trainable=True
        )
        super().build(input_shape)

    def call(self, inputs):
        """
        inputs: tuple  (embeddings [B, D],  labels [B, 1])
        returns: logits [B, num_classes]  → feed to SparseCategoricalCE
        """
        emb, labels = inputs

        # labels arrive as [B, 1] from the Input layer;
        # tf.one_hot needs [B] so squeeze the trailing dim
        labels = tf.squeeze(labels, axis=-1)                # [B, 1] → [B]

        # normalise class weights  →  unit vectors
        W_norm = tf.nn.l2_normalize(self.W, axis=1)         # [C, D]

        # cosine similarity matrix
        cosine = tf.matmul(emb, W_norm, transpose_b=True)   # [B, C]
        sine   = tf.sqrt(tf.clip_by_value(1.0 - cosine**2, 1e-8, 1.0))

        # cos(θ + m) = cos θ · cos m  −  sin θ · sin m
        phi = cosine * self.cos_m - sine * self.sin_m

        # easy-margin: guard when cos θ < th  (i.e. θ + m > π)
        phi = tf.where(cosine > self.th, phi, cosine - self.mm)

        # one-hot: replace target column with margin-adjusted logit
        one_hot = tf.one_hot(tf.cast(labels, tf.int32), self.num_classes)   # [B, C]
        output  = one_hot * phi + (1.0 - one_hot) * cosine
        output  = output * self.s                            # scale
        return output

    def get_config(self):
        cfg = super().get_config()
        cfg.update({
            'num_classes': self.num_classes, 'emb_dim': self.emb_dim,
            'scale': self.s, 'margin': self.m
        })
        return cfg

print("ArcFaceLayer defined.")

In [ ]:
# ============================================================
# CELL 8  –  Full training model  (embedding + ArcFace head)
# ============================================================

def build_training_model(num_classes):
    """
    Inputs  : face_input  [B, 112, 112, 3]
              label_input [B, 1]
    Output  : logits      [B, num_classes]

    At inference time use `embedding_model` directly.
    """
    face_input  = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='face_input')
    label_input = tf.keras.layers.Input(shape=(1,), dtype=tf.int32, name='label_input')

    emb    = embedding_model(face_input)                 # [B, 512]
    logits = ArcFaceLayer(
        num_classes=num_classes,
        emb_dim=EMB_DIM,
        scale=ARC_SCALE,
        margin=ARC_MARGIN,
        name='arcface'
    )([emb, label_input])                                # [B, C]

    train_model = tf.keras.Model(
        inputs=[face_input, label_input],
        outputs=logits,
        name='arcface_training'
    )
    return train_model

train_model = build_training_model(NUM_CLASSES)
train_model.summary(expand_nested=False)

# ── Freeze backbone initially (2 epochs warm-up) ──
embedding_model.trainable = False
print("\nBackbone frozen for warm-up. Will unfreeze after epoch 2.")

In [ ]:
# ============================================================
# CELL 9  –  Custom training loop with warm-up unfreeze
# ============================================================
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tqdm import tqdm
import time

WARMUP_EPOCHS = 2  # freeze backbone for this many epochs

# ── Loss ──
loss_fn = SparseCategoricalCrossentropy(from_logits=True)

# ── Initial optimiser (head only) ──
optimiser = AdamW(learning_rate=LR_INIT, weight_decay=WEIGHT_DECAY)
train_model.compile(optimizer=optimiser, loss=loss_fn)

# ── LR schedule: cosine decay ──
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
cosine_sched = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=LR_INIT,
    decay_steps=EPOCHS * steps_per_epoch,
    alpha=LR_MIN / LR_INIT
)

# ── History storage ──
history = {'train_loss': [], 'val_loss': [], 'val_auc': [], 'lr': []}
best_auc = 0.0
patience_counter = 0

# ─── helper: run one epoch ───
def run_epoch(epoch_idx):
    global best_auc, patience_counter

    epoch_start = time.time()

    # ── Warm-up unfreeze logic ──
    if epoch_idx == WARMUP_EPOCHS:
        print("→ Unfreezing LAST ResNet block only & freezing BN …")

    for layer in backbone.layers:
        layer.trainable = layer.name.startswith("conv5")

    for layer in backbone.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

    # ── Recompile with cosine LR ──
    train_model.compile(
        optimizer=AdamW(
            learning_rate=cosine_sched,
            weight_decay=WEIGHT_DECAY
        ),
        loss=loss_fn
    )

    # ── TRAIN ──
    epoch_losses = []
    pbar = tqdm(
        train_ds,
        total=steps_per_epoch,
        desc=f"Train Epoch {epoch_idx+1}/{EPOCHS}",
        unit="batch",
        ncols=110
    )

    for step, (imgs, labels) in enumerate(pbar):
        label_inp = tf.expand_dims(labels, -1)
        loss_val = train_model.train_on_batch([imgs, label_inp], labels)
        epoch_losses.append(loss_val)

        # current LR (schedule-safe)
        lr = train_model.optimizer.learning_rate
        current_lr = (
            float(lr(train_model.optimizer.iterations))
            if isinstance(lr, tf.keras.optimizers.schedules.LearningRateSchedule)
            else float(tf.keras.backend.get_value(lr))
        )

        pbar.set_postfix({
            "loss": f"{loss_val:.4f}",
            "lr": f"{current_lr:.2e}"
        })

    avg_train_loss = float(np.mean(epoch_losses))
    history['train_loss'].append(avg_train_loss)
    history['lr'].append(current_lr)

    # ── VAL LOSS ──
    val_losses = []
    for imgs, labels in val_ds:
        label_inp = tf.expand_dims(labels, -1)
        vl = train_model.evaluate([imgs, label_inp], labels, verbose=0)
        val_losses.append(vl)

    avg_val_loss = float(np.mean(val_losses))
    history['val_loss'].append(avg_val_loss)

    # ── VAL AUC ──
    auc_score = compute_val_auc(n_pairs=5000)
    history['val_auc'].append(auc_score)

    elapsed = (time.time() - epoch_start) / 60.0

    print(
        f"  Ep {epoch_idx+1:02d} | "
        f"train_loss={avg_train_loss:.4f} | "
        f"val_loss={avg_val_loss:.4f} | "
        f"val_AUC={auc_score:.4f} | "
        f"lr={current_lr:.2e} | "
        f"time={elapsed:.1f} min"
    )

    # ── Checkpoint ──
    if auc_score > best_auc:
        best_auc = auc_score
        patience_counter = 0
        embedding_model.save_weights(os.path.join(SAVE_DIR, 'best_embedding.weights.h5'))
        train_model.save_weights(os.path.join(SAVE_DIR, 'best_train_model.weights.h5'))
        print(f"  ✓ New best AUC → {best_auc:.4f}")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            return False  # early stop

    return True

print("✓ Training loop fixed and ready. Run Cell 11.")


In [ ]:
# ============================================================
# CELL 10  –  compute_val_auc()  — used inside the training loop
# ============================================================

def extract_all_embeddings(dataset, model):
    """Push every batch through the embedding model (no ArcFace)."""
    all_emb, all_lbl = [], []
    for imgs, labels in dataset:
        emb = model(imgs, training=False)            # BN uses moving stats
        all_emb.append(emb.numpy())
        all_lbl.append(labels.numpy())
    return np.concatenate(all_emb), np.concatenate(all_lbl)


def compute_val_auc(n_pairs=5000):
    """
    1. Extract embeddings for ALL val images.
    2. Sample n_pairs positive + n_pairs negative pairs.
    3. Return ROC-AUC on cosine similarities.
    """
    embs, labels = extract_all_embeddings(val_ds, embedding_model)

    # group indices by identity
    id2idx = {}
    for i, lbl in enumerate(labels):
        id2idx.setdefault(int(lbl), []).append(i)
    ids = list(id2idx.keys())

    sims, gts = [], []

    # ── positive pairs (same identity) ──
    for _ in range(n_pairs):
        lbl  = random.choice(ids)
        i, j = random.sample(id2idx[lbl], 2)
        sims.append(float(embs[i] @ embs[j]))
        gts.append(1)

    # ── negative pairs (different identities) ──
    for _ in range(n_pairs):
        a, b = random.sample(ids, 2)
        i    = random.choice(id2idx[a])
        j    = random.choice(id2idx[b])
        sims.append(float(embs[i] @ embs[j]))
        gts.append(0)

    return roc_auc_score(gts, sims)

print("compute_val_auc() defined.")

In [ ]:
# ============================================================
# CELL 11  –  Execute training
# ============================================================
print(f"Starting training … | Epochs={EPOCHS} | Batch={BATCH_SIZE} | Patience={PATIENCE}")
print(f"Train images : {len(train_files)}   Val images : {len(val_files)}\n")

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    should_continue = run_epoch(epoch)
    if not should_continue:
        print(f"\n⏹ Early stopping at epoch {epoch+1}. Best val AUC = {best_auc:.4f}")
        break
else:
    print(f"\n✓ Training complete. Best val AUC = {best_auc:.4f}")

# ── reload best weights ──
embedding_model.load_weights(os.path.join(SAVE_DIR, 'best_embedding.weights.h5'))
print("Best embedding weights restored.")

In [ ]:
# ============================================================
# CELL 12  –  Full evaluation  (AUC, EER, TAR@FAR, Accuracy)
# ============================================================

N_EVAL_PAIRS = 20_000                                  # per class (pos / neg)

# ── 1. Extract embeddings ──
val_embs, val_lbls = extract_all_embeddings(val_ds, embedding_model)
print(f"Extracted {val_embs.shape[0]} embeddings  dim={val_embs.shape[1]}")

# ── 2. Sample pairs ──
id2idx = {}
for i, lbl in enumerate(val_lbls):
    id2idx.setdefault(int(lbl), []).append(i)
ids = list(id2idx.keys())

sims, gts = [], []
for _ in range(N_EVAL_PAIRS):          # positives
    lbl  = random.choice(ids)
    i, j = random.sample(id2idx[lbl], 2)
    sims.append(float(val_embs[i] @ val_embs[j]))
    gts.append(1)
for _ in range(N_EVAL_PAIRS):          # negatives
    a, b = random.sample(ids, 2)
    i    = random.choice(id2idx[a])
    j    = random.choice(id2idx[b])
    sims.append(float(val_embs[i] @ val_embs[j]))
    gts.append(0)

sims = np.array(sims)
gts  = np.array(gts)

# ── 3. ROC ──
fpr, tpr, thresholds = roc_curve(gts, sims)
auc_score = roc_auc_score(gts, sims)

# ── 4. EER ──
fnr = 1 - tpr
abs_diff = np.abs(fnr - fpr)
eer_idx  = np.argmin(abs_diff)
eer       = (fpr[eer_idx] + fnr[eer_idx]) / 2.0
eer_thresh = thresholds[eer_idx]

# ── 5. TAR @ FAR ──
def tar_at_far(target_far):
    idx = np.argmin(np.abs(fpr - target_far))
    return tpr[idx]

tar_1e4 = tar_at_far(1e-4)
tar_1e3 = tar_at_far(1e-3)

# ── 6. Verification accuracy at EER threshold ──
preds = (sims >= eer_thresh).astype(int)
ver_acc = np.mean(preds == gts)

# ── Print summary ──
print("\n" + "="*48)
print("        EVALUATION SUMMARY  (108 unseen IDs)")
print("="*48)
print(f"  ROC-AUC              :  {auc_score:.6f}")
print(f"  EER                 :  {eer*100:.2f} %")
print(f"  EER threshold       :  {eer_thresh:.4f}")
print(f"  TAR @ FAR=1e-4      :  {tar_1e4:.4f}")
print(f"  TAR @ FAR=1e-3      :  {tar_1e3:.4f}")
print(f"  Verification Acc    :  {ver_acc*100:.2f} %")
print("="*48)

# ── persist for plotting ──
eval_results = dict(fpr=fpr, tpr=tpr, thresholds=thresholds,
                    auc=auc_score, eer=eer, eer_thresh=eer_thresh,
                    tar_1e4=tar_1e4, tar_1e3=tar_1e3, ver_acc=ver_acc,
                    sims=sims, gts=gts)

In [ ]:
# ============================================================
# CELL 13  –  Plot 1  →  ROC Curve  +  EER annotation
# ============================================================
fig, ax = plt.subplots(figsize=(8, 6.5))
fig.patch.set_facecolor('#0e0f14')
ax.set_facecolor('#16171e')

# ── ROC curve ──
ax.plot(eval_results['fpr'], eval_results['tpr'],
       color='#5b8cff', linewidth=2.2, label=f"AUC = {eval_results['auc']:.4f}")

# ── diagonal (random) ──
ax.plot([0, 1], [0, 1], '--', color='#3a3c4a', linewidth=1)

# ── EER point ──
eer_val = eval_results['eer']
ax.scatter([eer_val], [1 - eer_val], s=80, color='#f06482', zorder=5, edgecolors='#0e0f14', linewidths=2)
ax.annotate(
    f"EER = {eer_val*100:.2f} %",
    xy=(eer_val, 1 - eer_val),
    xytext=(eer_val + 0.06, 1 - eer_val - 0.08),
    fontsize=11, color='#f06482', fontweight='bold',
    arrowprops=dict(arrowstyle='->', color='#f06482')
)

# ── TAR @ FAR markers ──
for far_target, label_t, clr in [
    (1e-4, 'FAR=1e-4', '#34e8b0'),
    (1e-3, 'FAR=1e-3', '#ffc15e')
]:
    idx = np.argmin(np.abs(eval_results['fpr'] - far_target))
    ax.scatter([eval_results['fpr'][idx]], [eval_results['tpr'][idx]],
            s=60, color=clr, zorder=5, edgecolors='#0e0f14', linewidths=1.5)
    ax.annotate(f"{label_t}\nTAR={eval_results['tpr'][idx]:.3f}",
            xy=(eval_results['fpr'][idx], eval_results['tpr'][idx]),
            xytext=(eval_results['fpr'][idx] + 0.04, eval_results['tpr'][idx] - 0.06),
            fontsize=9, color=clr,
            arrowprops=dict(arrowstyle='->', color=clr))

# ── styling ──
ax.set_xlabel('False Positive Rate', fontsize=12, color='#8b90a0')
ax.set_ylabel('True Positive Rate', fontsize=12, color='#8b90a0')
ax.set_title('ROC Curve — Face Verification (108 Unseen Identities)',
           fontsize=14, color='#fff', pad=14)
ax.tick_params(colors='#8b90a0')
for sp in ax.spines.values(): sp.set_color('#2a2c38')
leg = ax.legend(loc='lower right', fontsize=11,
              facecolor='#1e1f2b', edgecolor='#2a2c38', labelcolor='#cdd1dc')
ax.set_xlim(-0.02, 1.02); ax.set_ylim(-0.02, 1.02)
ax.grid(True, color='#2a2c38', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'plot_roc.png'), dpi=150)
plt.show()

In [ ]:
# ============================================================
# CELL 14  –  Plot 2  →  Training / Val Loss  +  Val AUC  +  LR
# ============================================================
epochs_ran = len(history['train_loss'])
x_ax = np.arange(1, epochs_ran + 1)

fig, axes = plt.subplots(2, 1, figsize=(10, 7), gridspec_kw={'hspace': 0.35})
fig.patch.set_facecolor('#0e0f14')

# ── top: losses ──
ax = axes[0]
ax.set_facecolor('#16171e')
ax.plot(x_ax, history['train_loss'], color='#5b8cff', lw=2, label='Train Loss')
ax.plot(x_ax, history['val_loss'],   color='#f06482', lw=2, label='Val Loss')
ax.axvline(WARMUP_EPOCHS, color='#ffc15e', ls='--', lw=1, alpha=0.7)
ax.text(WARMUP_EPOCHS + 0.3, max(history['train_loss']) * 0.95,
       'backbone unfrozen', color='#ffc15e', fontsize=9)
ax.set_ylabel('Loss', color='#8b90a0', fontsize=11)
ax.set_title('Training Dynamics', color='#fff', fontsize=14, pad=10)
ax.tick_params(colors='#8b90a0')
for sp in ax.spines.values(): sp.set_color('#2a2c38')
ax.legend(facecolor='#1e1f2b', edgecolor='#2a2c38', labelcolor='#cdd1dc')
ax.grid(True, color='#2a2c38', ls='--', alpha=0.4)

# ── bottom-left: val AUC ──
ax2 = axes[1]
ax2.set_facecolor('#16171e')
ax2.plot(x_ax, history['val_auc'], color='#34e8b0', lw=2.2, label='Val AUC-ROC')
ax2.axhline(best_auc, color='#34e8b0', ls=':', lw=1, alpha=0.5)
ax2.text(1, best_auc + 0.002, f'best = {best_auc:.4f}', color='#34e8b0', fontsize=9)
ax2.set_ylabel('AUC-ROC', color='#8b90a0', fontsize=11)
ax2.set_xlabel('Epoch', color='#8b90a0', fontsize=11)
ax2.tick_params(colors='#8b90a0')
for sp in ax2.spines.values(): sp.set_color('#2a2c38')
ax2.legend(facecolor='#1e1f2b', edgecolor='#2a2c38', labelcolor='#cdd1dc')
ax2.grid(True, color='#2a2c38', ls='--', alpha=0.4)

# ── LR on a twin axis ──
ax3 = ax2.twinx()
ax3.plot(x_ax, history['lr'], color='#ffc15e', lw=1.2, ls='--', label='LR')
ax3.set_ylabel('Learning Rate', color='#ffc15e', fontsize=10)
ax3.tick_params(colors='#ffc15e')
ax3.set_facecolor('#16171e')
for sp in ax3.spines.values(): sp.set_color('#2a2c38')
lines3, labels3 = ax3.get_legend_handles_labels()
ax2.legend(ax2.get_legend_handles_labels()[0] + lines3,
           ax2.get_legend_handles_labels()[1] + labels3,
           facecolor='#1e1f2b', edgecolor='#2a2c38', labelcolor='#cdd1dc')

plt.savefig(os.path.join(SAVE_DIR, 'plot_history.png'), dpi=150)
plt.show()

In [ ]:
# ============================================================
# CELL 15  –  Plot 3  →  Cosine-similarity distributions
# ============================================================
pos_sims = eval_results['sims'][eval_results['gts'] == 1]
neg_sims = eval_results['sims'][eval_results['gts'] == 0]

fig, ax = plt.subplots(figsize=(9, 5))
fig.patch.set_facecolor('#0e0f14')
ax.set_facecolor('#16171e')

bins = np.linspace(-0.4, 1.0, 120)
ax.hist(neg_sims, bins=bins, density=True, alpha=0.55,
       color='#f06482', label='Negative (diff identity)', edgecolor='none')
ax.hist(pos_sims, bins=bins, density=True, alpha=0.55,
       color='#34e8b0', label='Positive (same identity)', edgecolor='none')

# ── EER threshold line ──
ax.axvline(eval_results['eer_thresh'], color='#ffc15e', ls='--', lw=2)
ax.text(eval_results['eer_thresh'] + 0.01, ax.get_ylim()[1] * 0.9,
       f"threshold = {eval_results['eer_thresh']:.3f}", color='#ffc15e', fontsize=10)

ax.set_xlabel('Cosine Similarity', color='#8b90a0', fontsize=12)
ax.set_ylabel('Density',           color='#8b90a0', fontsize=12)
ax.set_title('Pair-wise Similarity: Positive vs Negative', color='#fff', fontsize=14, pad=12)
ax.tick_params(colors='#8b90a0')
for sp in ax.spines.values(): sp.set_color('#2a2c38')
ax.legend(facecolor='#1e1f2b', edgecolor='#2a2c38', labelcolor='#cdd1dc', fontsize=11)
ax.grid(True, color='#2a2c38', ls='--', alpha=0.4)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'plot_sim_dist.png'), dpi=150)
plt.show()

In [ ]:
# ============================================================
# CELL 16  –  Plot 4  →  UMAP cluster visualisation
# ============================================================
# !pip install umap-learn   ← uncomment if needed on Kaggle
import umap

# ── subsample for speed (optional: use all val_embs) ──
MAX_PER_ID  = 30                                    # images per identity to plot
subset_idx  = []
for lbl, idxs in id2idx.items():
    subset_idx.extend(random.sample(idxs, min(MAX_PER_ID, len(idxs))))
subset_embs = val_embs[subset_idx]
subset_lbls = val_lbls[subset_idx]

# ── UMAP projection ──
reducer = umap.UMAP(
    n_components=2, n_neighbors=15,
    min_dist=0.08, metric='cosine', random_state=SEED
)
coords = reducer.fit_transform(subset_embs)
print(f"UMAP done  →  {coords.shape}")

# ── Plot ──
fig, ax = plt.subplots(figsize=(11, 9))
fig.patch.set_facecolor('#0e0f14')
ax.set_facecolor('#10111a')

# ── colour map: use enough distinct colours ──
n_unique = len(np.unique(subset_lbls))
cmap     = plt.cm.get_cmap('tab20', n_unique)

scatter = ax.scatter(
    coords[:, 0], coords[:, 1],
    c=subset_lbls, cmap=cmap, s=18, alpha=0.75,
    edgecolors='#0e0f14', linewidths=0.6
)

ax.set_xlabel('UMAP-1', color='#8b90a0', fontsize=11)
ax.set_ylabel('UMAP-2', color='#8b90a0', fontsize=11)
ax.set_title(
    f'Embedding Space — {len(np.unique(subset_lbls))} Unseen Identities  '
    f'(UMAP · cosine)',
    color='#fff', fontsize=15, pad=14
)
ax.tick_params(colors='#8b90a0')
for sp in ax.spines.values(): sp.set_color('#2a2c38')
ax.grid(True, color='#1e1f2b', ls='-', alpha=0.35)

# ── legend-free note ──
ax.text(ax.get_xlim()[0] + 0.2, ax.get_ylim()[0] + 0.3,
       'Each colour = one identity', color='#6b7280', fontsize=10, style='italic')

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'plot_umap.png'), dpi=150)
plt.show()

In [ ]:
# ============================================================
# CELL 17  –  Plot 5  →  DET curve  (log-log FPR vs FNR)
# ============================================================
fpr_det = eval_results['fpr']
fnr_det = 1 - eval_results['tpr']

# clip to avoid log(0)
eps      = 1e-5
fpr_clip = np.clip(fpr_det, eps, 1)
fnr_clip = np.clip(fnr_det, eps, 1)

fig, ax = plt.subplots(figsize=(7.5, 6))
fig.patch.set_facecolor('#0e0f14')
ax.set_facecolor('#16171e')

ax.loglog(fpr_clip, fnr_clip, color='#5b8cff', linewidth=2.2)

# ── EER marker ──
ax.scatter([eer_val], [eer_val], s=70, color='#f06482',
           zorder=5, edgecolors='#0e0f14', linewidths=2)
ax.annotate(f'EER = {eer_val*100:.2f} %', xy=(eer_val, eer_val),
            xytext=(eer_val * 4, eer_val * 0.25),
            color='#f06482', fontsize=11, fontweight='bold',
            arrowprops=dict(arrowstyle='->', color='#f06482'))

# ── random baseline ──
ax.loglog([eps, 1], [eps, 1], '--', color='#3a3c4a', lw=1, label='Random')

ax.set_xlabel('False Positive Rate', color='#8b90a0', fontsize=11)
ax.set_ylabel('False Negative Rate', color='#8b90a0', fontsize=11)
ax.set_title('DET Curve', color='#fff', fontsize=14, pad=12)
ax.tick_params(colors='#8b90a0', which='both')
for sp in ax.spines.values(): sp.set_color('#2a2c38')
ax.legend(facecolor='#1e1f2b', edgecolor='#2a2c38', labelcolor='#cdd1dc')
ax.grid(True, color='#2a2c38', which='both', ls='--', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'plot_det.png'), dpi=150)
plt.show()

In [ ]:
# ============================================================
# CELL 18  –  Plot 6  →  Metric summary card  (bar chart)
# ============================================================
metrics = {
    'AUC-ROC':       eval_results['auc'],
    'Verify Acc':    eval_results['ver_acc'],
    'TAR@FAR1e-3':  eval_results['tar_1e3'],
    'TAR@FAR1e-4':  eval_results['tar_1e4'],
    '1 − EER':       1 - eval_results['eer'],
}

fig, ax = plt.subplots(figsize=(9, 4))
fig.patch.set_facecolor('#0e0f14')
ax.set_facecolor('#16171e')

colors = ['#5b8cff', '#34e8b0', '#ffc15e', '#f06482', '#a78bfa']
bars   = ax.barh(list(metrics.keys()), list(metrics.values()),
            color=colors, height=0.45, edgecolor='none')

# ── value labels ──
for bar, val in zip(bars, metrics.values()):
    ax.text(val + 0.005, bar.get_y() + bar.get_height() / 2,
           f'{val*100:.2f} %', va='center', color='#fff', fontsize=12, fontweight='bold')

# ── target line at 1.0 ──
ax.axvline(1.0, color='#2a2c38', ls='--', lw=1)
ax.set_xlim(0.7, 1.04)
ax.set_xlabel('Score', color='#8b90a0', fontsize=11)
ax.set_title('Evaluation Metrics — 108 Unseen Identities', color='#fff', fontsize=14, pad=12)
ax.tick_params(colors='#8b90a0')
for sp in ax.spines.values(): sp.set_color('#2a2c38')
ax.grid(True, color='#2a2c38', ls='--', alpha=0.4, axis='x')

plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, 'plot_metrics.png'), dpi=150)
plt.show()

In [ ]:
# ============================================================
# CELL 19  –  Export the embedding model  (no ArcFace head)
# ============================================================

# ── 1. Keras SavedModel ──
saved_path = os.path.join(SAVE_DIR, 'face_embedding_savedmodel')
embedding_model.save(saved_path)
print(f"✓ SavedModel  →  {saved_path}")

# ── 2. H5 weights only (lightest) ──
h5_path = os.path.join(SAVE_DIR, 'face_embedding_weights.h5')
embedding_model.save_weights(h5_path)
print(f"✓ Weights H5  →  {h5_path}")

# ── 3. ONNX  (optional — uncomment if tf2onnx is available) ──
# !pip install tf2onnx onnx
# import tf2onnx, onnx
# spec = [tf.TensorSpec(shape=[1,112,112,3], dtype=tf.float32)]
# onnx_model, _ = tf2onnx.convert.from_keras(embedding_model, input_signature=spec)
# onnx.save(onnx_model, os.path.join(SAVE_DIR, 'face_embedding.onnx'))
# print("✓ ONNX model saved")

# ── 4. Quick inference smoke-test ──
dummy = np.random.randn(1, 112, 112, 3).astype(np.float32)
out   = embedding_model.predict(dummy)
print(f"\n✓ Smoke test → output shape: {out.shape}, L2-norm: {np.linalg.norm(out, axis=1)}")